In [1]:
!cat .env

STAGE=ENGIE_PRODUCTION


In [2]:
import json
from helpers.fields import FeatureCollectionField
import shapefile as shpfile
from tqdm.notebook import tqdm
from helpers.histories import bulk_update_with_history
from collections import defaultdict
import random

In [3]:
def get_uf(uf_name):
    if type(uf_name) != 'str':
        uf_code=str(uf_name)
    
    uf=[{'id': '12', 'name': 'AC'}, {'id': '27', 'name': 'AL' }, {'id': '13', 'name': 'AM'}, {'id': '16', 'name': 'AP'},
       {'id': '29', 'name': 'BA'}, {'id': '23', 'name': 'CE' }, {'id': '53', 'name': 'DF'}, {'id': '32', 'name': 'ES'},
       {'id': '52', 'name': 'GO'}, {'id': '21', 'name': 'MA' }, {'id': '31', 'name': 'MG'}, {'id': '50', 'name': 'MS'},
       {'id': '51', 'name': 'MT'}, {'id': '15', 'name': 'PA' }, {'id': '25', 'name': 'PB'}, {'id': '26', 'name': 'PE'},
       {'id': '22', 'name': 'PI'}, {'id': '41', 'name': 'PR' }, {'id': '33', 'name': 'RJ'}, {'id': '24', 'name': 'RN'},
       {'id': '11', 'name': 'RO'}, {'id': '14', 'name': 'RR' }, {'id': '43', 'name': 'RS'}, {'id': '42', 'name': 'SC'},
       {'id': '28', 'name': 'SE'}, {'id': '35', 'name': 'SP' }, {'id': '17', 'name': 'TO'}]
    
    uf_translation={item['name']: item['id'] for item in uf}

    return uf_translation[uf_name]

# Escolher unidade

In [4]:
company = Company.objects.filter(name__startswith='GSO')
engie=User.objects.get(username='engie')
company, engie

(<QuerySet [<Company: c85df77c-3e92-4f60-b9c1-de5866400d2d: GSO - UHE-CANA BRAVA>, <Company: 571581e3-da3a-469e-8676-0dc58c056ef7: GSO - UHE-SALTO OSORIO>, <Company: 4920816b-507a-48b1-86d5-d2eef6c63a1f: GSO - UTPS>, <Company: 37b4c80d-3f2b-4032-8cac-6ba1560d3144: GSO - UHE-SALTO SANTIAGO>, <Company: 1021dc11-d12c-4a27-88d5-dde0fad2be7d: GSO - UHE-PONTE DE PEDRA>, <Company: f0bdd035-6af9-401c-9233-26c5cdd808b0: GSO - UHE-JAGUARA>, <Company: 62ad80d9-b785-4037-933a-e23238728220: GSO - UHE-MIRANDA>, <Company: 5f0ef84c-9985-4519-95de-e87fb4fcea21: GSO - PCH-JOSÉ GELÁZIO DA ROCHA>, <Company: 1b1511d1-6f28-4aed-b02c-7c820c5f9dbb: GSO - PCH-RONDONÓPOLIS>, <Company: 33e4f910-db35-46ce-95c8-56cad68eeb94: GSO - UHE-ESTREITO>, <Company: 1f30f493-c653-46c7-b073-0046837347f7: GSO - UHE-SÃO SALVADOR>, <Company: 110bce05-5146-421e-a5a7-ffb32ee92a91: GSO - UHE-PASSO FUNDO>, <Company: e9fce210-f652-4d7d-befe-32e02a8f0f60: GSO - UHE-ITA>, <Company: 4b4bfdad-f8b8-403b-93a6-26e846a6770b: GSO - UHE-MACHAD

In [5]:
estados=ShapeFile.objects.filter(name__icontains='municipios')

In [6]:
estados.count(),list(estados.values_list('name',flat=True))

(10,
 ['GO_Municipios_2022',
  'MA_Municipios_2022',
  'MG_Municipios_2022',
  'MS_Municipios_2022',
  'MT_Municipios_2022',
  'PR_Municipios_2022',
  'RS_Municipios_2022',
  'SP_Municipios_2022',
  'TO_Municipios_2022',
  'SC_Municipios_2022'])

In [7]:
occs = OccurrenceType.objects.filter(
    name__in=['Sinalização da rota de fuga','Ponto de encontro','Ficha descrita da estação remota']
)
occs.count()

3

In [11]:
from tqdm.notebook import tqdm
for estado in estados:
    for cidades_props, cidades_geom in tqdm(zip(estado.properties,estado.geometry)):
        query = OccurrenceRecord.objects.filter(company__in=company,occurrence_type__in=occs,geometry__intersects=cidades_geom)
        if query.exists():
            print(query[0].company.name,query.count())
            for record in query:
                cidade_decoded = cidades_props['NM_MUN'].encode('iso-8859-1').decode('utf-8')
                if record.city is not None:
                    pass
                    if  record.city.name.lower() != cidade_decoded.lower():

                        try:
                            new_city=City.objects.filter(firm__company=record.company,name=cidade_decoded).first()
                        except Exception as e:
                            new_city=City(name=cidade_decoded,uf_code=get_uf(cidades_props['SIGLA_UF']))
                            new_city.save()
                        print(new_city,' | ',record.city.name,' | ',record.number)
                        record.uf_code = record.new_city.uf_code
                        record.city=new_city
                        record.save()
                else:
                    try:
                        new_city=City.objects.filter(firm__company=record.company,name=cidade_decoded).first()
                    except Exception as e:
                        new_city=City(name=cidade_decoded,uf_code=get_uf(cidades_props['SIGLA_UF']))
                        new_city.save()
                    print(new_city,' | ',record.city,' | ',record.number)
                    record.uf_code = record.new_city.uf_code
                    record.city=new_city
                    record.save()


0it [00:00, ?it/s]

GSO - UHE-CANA BRAVA 11
GSO - UHE-CANA BRAVA 17


0it [00:00, ?it/s]

GSO - UHE-ESTREITO 116


0it [00:00, ?it/s]

...

GSO - UHE-JAGUARA 11


0it [00:00, ?it/s]

GSO - UHE-ESTREITO 58
GSO - UHE-SÃO SALVADOR 33
GSO - UHE-SÃO SALVADOR 70


0it [00:00, ?it/s]

GSO - UHE-ITA 95
[42] Itá  |  None  |  UHIT-PE-2023.0035
GSO - UHE-MACHADINHO 25
[42] Piratuba  |  None  |  UHMA-PE-2023.0000


In [14]:
for estado in estados:
    for cidades_props, cidades_geom in tqdm(zip(estado.properties,estado.geometry)):
        query = OccurrenceRecord.objects.filter(company__in=company,occurrence_type__in=occs,geometry__intersects=cidades_geom)
        if query.exists():
            print(query[0].company.name,query.count())
            for record in query:
                cidade_decoded = cidades_props['NM_MUN'].encode('iso-8859-1').decode('utf-8')
                if record.uf_code == None:
                    record.uf_code = record.city.uf_code
                    record.save()
                    

0it [00:00, ?it/s]

GSO - UHE-CANA BRAVA 11
GSO - UHE-CANA BRAVA 17


0it [00:00, ?it/s]

GSO - UHE-ESTREITO 116


0it [00:00, ?it/s]

...

GSO - UHE-JAGUARA 11


0it [00:00, ?it/s]

GSO - UHE-ESTREITO 58
GSO - UHE-SÃO SALVADOR 33
GSO - UHE-SÃO SALVADOR 70


0it [00:00, ?it/s]

GSO - UHE-ITA 95
GSO - UHE-MACHADINHO 25


In [17]:
cidades_props[]

{'CD_MUN': '4220000',
 'NM_MUN': 'BalneÃ¡rio RincÃ£o',
 'AREA_KM2': 63.42,
 'SIGLA_UF': 'SC'}